In [25]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing
import sklearn.decomposition
import sklearn.model_selection


In [26]:
# load data
def load_data(filepath):
    data = pd.read_csv(filepath)
    return data

df = load_data("./dataset/CLAMP_Train.csv")

In [38]:
df.head()

,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_csum,...,CheckSum,Subsystem,DllCharacteristics,SizeOfStackReserve,SizeOfStackCommit,SizeOfHeapReserve,SizeOfHeapCommit,LoaderFlags,NumberOfRvaAndSizes,class
0,23117,144,3,0,4,0,65535,0,184,0,...,65452,2,1024,1048576,4096,1048576,4096,0,16,0
1,23117,144,3,0,4,0,65535,0,184,0,...,0,2,0,1048576,4096,1048576,4096,0,16,1
2,23117,144,3,0,4,0,65535,0,184,0,...,119291,2,0,1048576,4096,1048576,4096,0,16,0
3,23117,144,3,0,4,0,65535,0,184,0,...,91168,3,320,262144,4096,1048576,4096,0,16,0
4,23117,144,3,0,4,0,65535,0,184,0,...,82656,2,32768,1048576,4096,1048576,4096,0,16,1


In [28]:
def find_dataset_statistics(dataset:pd.DataFrame,target_col:str) -> tuple[int,int,int,int,float]:

    #Total number of records
    n_records = dataset.shape[0]
    #Total number of columns 
    n_columns = len(dataset.columns)
    #Number of records where target is negative
    n_negative = dataset[target_col].value_counts()[0]
    #Number of records where where target is positive
    n_positive = dataset[target_col].value_counts()[1]
    # Percentage of instances of positive target value
    perc_positive =  (n_positive/n_records)*100

    return n_records,n_columns,n_negative,n_positive,perc_positive

In [33]:

def train_test_split(  dataset: pd.DataFrame,
                       target_col: str, 
                       test_size: float,
                       stratify: bool,
                       random_state: int) -> tuple[pd.DataFrame,pd.DataFrame,pd.Series,pd.Series]:
    # get features from dataset
    X = dataset.drop(target_col, axis=1)
    # get the target
    y = dataset[target_col]
    # split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=stratify, random_state=random_state)

    train_features = X_train
    test_features = X_test
    train_targets = y_train
    test_targets = y_test

    return train_features,test_features,train_targets,test_targets



(10, 5, 5, 5, 50.0)